# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Traitement Naturel du Langage (NLP) : Génération de Texte avec des Réseaux Récurrent. 

Au cours de ce calepin, nous allons voir comment générer des description de produits à l'aide de Réseaux Récurents et notamment grace aux structure LSTM (Long-Short Term Memory). 

L'intérêt de cette application est limité. Les descriptions de textes de ce document sont trop pauvres syntaxiquement pour pouvoir juger réellement de la qualité du texte généré. L'intérêt réel de ce calepin est de voir comment les données doivent être mis en forme pour être utilisé dans un réseau recurrent dans un but de génération de texte.

## Librairies

In [1]:
#Importation des librairies utilisées

import pandas as pd
import numpy as np
import pickle
import functools
from tqdm import tqdm

import tensorflow.keras.models as km
import tensorflow.keras.layers as kl

/Users/brendanguillouet/anaconda3/envs/INSA/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/brendanguillouet/anaconda3/envs/INSA/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/brendanguillouet/anaconda3/envs/INSA/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/brendanguill

## Téléchargement des données

La Catégorie de Niveau 3 `COQUE - BUMPER - FACADE TELEPHONE` est la catégorie le plus représenté du jeu de données originale **Cdiscount** avec  2.184.671 déscriptions présentent.  Parmis ces descriptions, 1.761.637 sont composés d'exactement 197 caractères. 

Nous allons nous servir de ces lignes (ou un sous ensemble de ces lignes, en fonction de la puissance de calcul disponible sur votre machine) pour apprendre un modèle de génération de texte qui permettra de généré automatiquement une nouvelle description de ce type de produit.

In [2]:
N = 100000
DATA_DIR = ""
X = np.load(DATA_DIR+"data/description_coque.npy")[:N]
print(X.shape)
print(X[:3])

(100000,)
["Pour apple iphone 4 : coque bumper silicone blanc - Cet étui en silicone rigide protège et habille votre APPLE iPhone 4. Parfaitement adapté, il permet l'accès à toutes les fo… Voir la présentation"
 "Pour htc one x : coque noire rigide - Cette coque protège et habille avec sobriété votre HTC ONE X. Parfaitement adaptée, elle permet l'accès à toutes les fonctionnalités de v… Voir la présentation"
 "Pour htc one x : coque blanche rigide - Cette coque protège et habille avec sobriété votre HTC ONE X. Parfaitement adaptée, elle permet l'accès à toutes les fonctionnalités de… Voir la présentation"]


**Exercice** Vérifiez que toutes les séquences sont bien de tailles 197.

In [3]:
Nd=197

## Mise en forme  des données

La génération de texte implique de constuire un réseau `Many-To-One` :

<img src="https://raw.githubusercontent.com/wikistat/AI-Frameworks/master/slides/OneToMany.png" alt="drawing" width="400"/>

Ou la prédiction $y_t$ servira d'entrée au réseau au temps $t+1$, i.e : $y_t=x_{t+1}$. 

Chaque $x_t$ représente ici un caractère de la déscription encodé en One-Hot encoding. Ainsi une description $x$ composé de $N_d$ caractères sera modélisé par une matrice de taille $(N_v\times N_d)$  $x=[x_1,x_2,...,x_{N_d}]$  ou $x_i \in \mathbb{R}^{N_v}$

### Création de la liste des caractères

Afin d'encoder les description sous format 'One-Hot encoding'  nous devons dans un premier temps retrouver la taille $Nv$ de notre vocabulaire constitué de tout les caractères présent dans la description.

In [4]:
chars = list(functools.reduce(lambda x,y : x.union(y), [set(x) for x in X], set()))
print("Vocabulaire : " +  str(chars))

Vocabulaire : ['M', 'k', ':', '+', 'f', '3', 'T', 's', 'y', 'â', 'v', 'h', 'é', ',', 'E', '%', '7', '/', 'q', '!', '(', 'J', '9', 'ô', 'j', 'Z', 'a', 'I', 'z', 't', '0', '&', '?', 'b', 'B', 'g', '"', 'K', 'i', ' ', 'd', 'l', 'm', 'P', 'p', 'x', 'u', 'o', 'H', ')', 'è', 'V', 'R', '8', 'n', '\xa0', "'", 'Q', 'ç', 'ê', '…', 'F', 'à', '5', '6', 'X', 'e', 'G', 'c', 'w', 'A', '-', 'U', '.', 'O', '4', '*', 'L', 'D', 'r', '2', 'Y', 'N', '1', 'C', 'S', 'W']


Nous ajoutons à ce vocabulaire deux indicateur permettant de localiser le début et la fin de chaque description

In [ ]:
chars.extend(["start","end"])

In [6]:
Nv = len(chars)
print("Taille du vocabulaire : %d" %Nv)

Taille du vocabulaire : 89


### Création des dictionnaires

Les dictionnaires `char_to_int` et `int_to_char` permettent respectivement d'encoder une description texte et de décoder un encodage `One-Hot``

In [7]:
int_to_char = {i:c for i,c in enumerate(chars)}
char_to_int = {c:i for i,c in int_to_char.items()}
I_START = char_to_int["start"]
I_END = char_to_int["end"]

### Encodage des Descriptions


La fonction suivante, permet d'encoder une matrice $X\in \mathbb{R}^{N \times N_d}$ constitués de *N* descriptions en une matrice $X_{vec} \in \mathbb{R}^{N \times N_d \times N_v}$ contenant les description encodées.

In [8]:
def encode_input_output_sequence(x, length_sequence, size_vocab, char_to_int_dic, i_start, i_end):
    n = x.shape[0]
    x_vec = np.zeros((n,length_sequence, size_vocab))
    y_vec = np.zeros((n,length_sequence, size_vocab))
    x_vec[:,0,i_start] = 1
    y_vec[:,-1,i_end] = 1
    for ix,x in tqdm(enumerate(x)):
        for ic,c in enumerate(x):
            c_int = char_to_int_dic[c]
            x_vec[ix,ic+1,c_int]=1
    y_vec[:,:-1,:] = x_vec[:,1:,:] 
    return x_vec, y_vec


In [9]:
X_vec, Y_vec = encode_input_output_sequence(X[:N], Nd+1, Nv, char_to_int,I_START,I_END)

100000it [00:08, 11186.37it/s]


In [10]:
X_vec.shape

(100000, 198, 89)

**Exercice** Retrouvez la phrase originale de la phrase test affiché ci-dessous à partir de la phrase encodé. Vérifiez que x et y sont bien les mêmes descriptions seulement décalées d'un index

In [11]:
# %load solution/3_1.py

## Apprentissage

Nous allons maintenant définir notre modèle récurrent afin de générer notre modèle de prédiction. 

Prenez le temps de bien comprendre toutes les fonctions et arguments utilisés pour construire ce modèle.

In [12]:
nb_hidden = 32
epochs = 20
batch_size= 128

model = km.Sequential()
model.add(kl.LSTM(nb_hidden, input_shape=(None, Nv), return_sequences=True))
model.add(kl.TimeDistributed(kl.Dense(Nv)))
model.add(kl.Activation('softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 32)          15616     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 89)          2937      
_________________________________________________________________
activation (Activation)      (None, None, 89)          0         
Total params: 18,553
Trainable params: 18,553
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(X_vec, Y_vec, epochs=epochs, batch_size=batch_size)
model.save("data/generate_model.h5")

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
100000/100000 [==============================] - 188s 2ms/sample - loss: 2.6827
Epoch 2/20
100000/100000 [==============================] - 170s 2ms/sample - loss: 1.3799
Epoch 3/20
 94848/100000 [===========================>..] - ETA: 8s - loss: 0.8067

**Q** Pourquoi est-ce la `categorical_crossentropy` qui est utilisée comme fonction de perte?

## Génération de Texte

La celulle suivante permet de générer une description produit :

In [ ]:
x_pred = np.zeros((1, Nd+1, Nv))
print("step 0")
x_pred[0,0,I_START] =1
x_pred_str = decode_sequence(x_pred[0], int_to_char)
print(x_pred_str)

for i in range(Nd):
    ix = np.argmax(model.predict(x_pred[:,:i+1,:])[0][-1,:])
    x_pred[0,i+1,ix] = 1
x_pred_str=decode_sequence(x_pred[0], int_to_char)
print(x_pred_str)

**Q** Comment cette génération est-elle produite? 

**Exercice** Effectuez une génération en choissisant la ou les premières lettres qui seront générées.

In [ ]:
#%load solution/3_2.py

**Exercice** Effectuez une génération en ajoutant de l'aléa. Vous pouvez par exemple faire en sorte que chaque lettre soit séléctionnée selon une loi multinomiale.

In [ ]:
#%load solution/3_3.py